In [ ]:
!pip install vaex
!pip install --upgrade vaex
!pip install dash
!pip install rtree 
!pip install pygeos
!pip install geopandas
!pip install folium matplotlib mapclassify geopandas -q -q -q
!pip install folium matplotlib mapclassify -q -q -q

In [ ]:
# Have to restart runtime for this to work
import vaex as vx
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import geopandas as gpd
import pygeos
import rtree
import folium
from branca.utilities import split_six

In [ ]:
# Load Datasets
min5= vx.open('/content/5min.parquet')   # 5 min data
metadata= pd.read_csv('metadata.csv') # Metadata
energy_consumption= pd.read_csv('/content/energy_consumption.csv') # energy_consumption

# Load data defining boundaries of UK Districts
url= 'http://geoportal1-ons.opendata.arcgis.com/datasets/01fd6b2d7600446d8af768005992f76a_4.geojson'
districts = gpd.read_file(url) 

In [ ]:
# clean energy consumption 
energy_consumption['region']= energy_consumption['region'].str.replace(' ','') 

# Cleaning disticts df
districts = districts[['nuts118nm','long','lat','geometry']]
districts['nuts118nm']= districts['nuts118nm'].str.replace('England','').str.replace('(','').str.replace(')','').str.replace('of','').str.replace(' ','') # clean renaming colum with region names
districts = districts.rename(columns={"nuts118nm": "region",
                                      "long":"district_long",
                                      "lat":"district_lat"}) 

In [ ]:
# Decomposing date
min5['day'],min5['month'],min5['year'] = min5.timestamp.dt.day, min5.timestamp.dt.month,min5.timestamp.dt.year,
min5['hour'],min5['minute'],min5['second'] = min5.timestamp.dt.hour,  min5.timestamp.dt.minute, min5.timestamp.dt.second
min5.head(2)

In [ ]:
ss_id = min5.ss_id.unique()
# Create a new dataframe with unique ss_id and info from metadata
ssid_df = pd.DataFrame(ss_id,columns=['ss_id'])
min5_meta  = ssid_df.merge(metadata, on='ss_id', how='left') # Merging 1 and 2
min5_meta = min5_meta.dropna()

In [ ]:
min5_meta_geo = gpd.GeoDataFrame(min5_meta,geometry=gpd.points_from_xy(min5_meta.longitude_rounded, 
                                                                  min5_meta.latitude_rounded))
pv_geo = gpd.sjoin(min5_meta_geo, districts, op='within')


# Part2: How does power generated vary over time

In [ ]:
plot1 = min5.groupby(['ss_id','year']).agg({'generation_wh': 'mean'}).to_pandas_df().sort_values('year', ascending=False)

plt.style.use('ggplot')
palette={2018:'#36454F',2019:'#36454F',2020:'#36454F',2021:'#fada5e' }
sns.barplot(x='year',y='generation_wh', data=plot1,hue='year', palette=palette)

# titles
plt.xlabel('Year') 
plt.ylabel('Average Power Generated')   
plt.title("Average Power Generated by Year")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

In [ ]:
# Number of pv systems operational in each year
plt2 = min5.groupby(min5.year).agg({'ss_id': 'count'}).to_pandas_df() 

plt.style.use('ggplot')
palette={2018:'#36454F',2019:'#36454F',2020:'#36454F',2021:'#fada5e' }

sns.barplot(data=plt2, x='year', y='ss_id',hue='year', palette=palette)

# titles
plt.xlabel('Year') 
plt.ylabel('Count of Pv Systems')   
plt.title("Count of Pv Systems by Year")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

### Plotting Average Energy Generation by Year and Month

In [ ]:
# grouping data by ss_id, year and month, and finding the average power generated
by_year_month = min5.groupby(['ss_id','year','month']).agg({'generation_wh': 'mean'}).to_pandas_df() 
# Combine data from table abouve with census data 
combined_month = by_year_month.merge(metadata, on='ss_id', how='left')
# creating a colum for date for the plotty animation filter  
combined_month["date"] = combined_month['year'].astype(str) +"-"+ combined_month["month"].astype(str)

In [ ]:
 #Creating a graph that will show power generation in the UK by year and month
import plotly.express as px

fig = px.scatter_geo(combined_month, lat="latitude_rounded", lon="longitude_rounded", color="generation_wh",
                     hover_name="ss_id", size="generation_wh",
                     animation_frame="date",
                     projection="natural earth",
                     scope ='europe',
                     color_continuous_scale=["blue", "purple", "red"],
                     center=dict(lat=combined_month.latitude_rounded.mean(), lon=combined_month.longitude_rounded.mean()))

fig.update_traces(marker=dict(size=10))
fig.update_layout(autosize=True,height=600,geo=dict(projection_scale=6))
fig.show()

### Plotting Average Energy Generation per Year by District

In [ ]:
# Consider averages by region
avg_region = min5.groupby(['ss_id','year','month','hour']).agg({'generation_wh': 'mean'}).to_pandas_df().set_index('ss_id')

# select only relevant columns
ssid_region= pv_geo[['ss_id','region',]].set_index('ss_id')

# join
region_date_gen = avg_region.join(ssid_region, on='ss_id', how='left').reset_index()
region_date_gen.head(2)

In [ ]:
region_date_gen_year = region_date_gen.groupby(['region','year']).agg({'generation_wh': 'mean'}).reset_index()

plt.figure(figsize=(40,20))
ax = sns.barplot(data=region_date_gen_year, 
                 x='year',
                 y='generation_wh',
                 hue='region',
                 palette=sns.color_palette('icefire'))

plt.xlabel('Year') 
plt.ylabel('Average Power Generated (wh)')   
plt.title("Average Power Generated (wh) by Year and District")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

### Plotting Average Energy Generation by Month for 2019

In [ ]:
# Selecting only 2019
region_date_gen_2019 = region_date_gen[region_date_gen['year']==2019]
region_date_gen_2019.head(2)

In [ ]:
#Power Generated (wh) by Month
plt.style.use('ggplot')

sns.barplot(data=region_date_gen_2019, 
            x='month', y='generation_wh',dodge=False,
            hue='month',palette=sns.color_palette('icefire'))

plt.xlabel('Month') 
plt.ylabel('Average Power Generated (wh)')   
plt.title("Average Power Generated (wh) by Month")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

In [ ]:
plt.figure(figsize=(40,20))
ax = sns.barplot(data=region_date_gen_2019, x='month', y='generation_wh',
                 dodge=False,
                 hue='month',palette=sns.color_palette('icefire'))

plt.xlabel('Month') 
plt.ylabel('Average Power Generated (wh)')   
plt.title("Average Power Generated (wh) by Month and District")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

In [ ]:
region_date_gen_2019.set_index('ss_id')
metadata = metadata.set_index('ss_id')# converting table to pd dataframe

# Combine data from table abouve with census data 
region_date_gen_loc_2019 = region_date_gen_2019.join(metadata, on='ss_id', how='left')
region_date_gen_loc_2019.head(2)

In [ ]:
#Creating a graph that will show Average power generation in the UK by month in 2019

fig = px.scatter_geo(region_date_gen_loc_2019, lat="latitude_rounded", lon="longitude_rounded", color="generation_wh",
                     hover_name="ss_id", size="generation_wh",
                     animation_frame="month",
                     projection="natural earth",
                     scope ='europe',
                     color_continuous_scale=["blue", "purple", "red"],
                     center=dict(lat=region_date_gen_loc_2019.latitude_rounded.mean(), lon=region_date_gen_loc_2019.longitude_rounded.mean()))

fig.update_traces(marker=dict(size=10))
fig.update_layout(autosize=True,height=600,geo=dict(projection_scale=6))
fig.show()

### Plotting Average Energy Generation by day for 2019

In [ ]:
# Power generated per hour by region (2019)
power_per_hour = region_date_gen_2019.groupby(['region','hour']).agg({'generation_wh': 'mean'}).reset_index()

plt.figure(figsize=(40,20))
ax = sns.barplot(data=power_per_hour, x='hour', y='generation_wh',hue='region', palette=sns.color_palette('icefire'))

plt.xlabel('Hour') 
plt.ylabel('Average Power Generated (wh)')   
plt.title("Average Power Generated (wh) by Hour and District")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

In [ ]:
#Creating a graph that will show power generation in the UK by Hour and month

fig = px.scatter_geo(region_date_gen_loc_2019, lat="latitude_rounded", lon="longitude_rounded", color="generation_wh",
                     hover_name="ss_id", size="generation_wh",
                     animation_frame="hour",
                     projection="natural earth",
                     scope ='europe',
                     color_continuous_scale=["blue", "purple", "red"],
                     center=dict(lat=region_date_gen_loc_2019.latitude_rounded.mean(), lon=region_date_gen_loc_2019.longitude_rounded.mean())
                     )

fig.update_traces(marker=dict(size=10))
fig.update_layout(autosize=True,height=600,geo=dict(projection_scale=6))
fig.show()